Evaluating the classification accuracy of BERT, BART, GPT, Llama models (which are politically leaning) on text classification accuracy of politically biased statements. 

In [9]:
import pandas as pd
df = pd.read_csv("data/IBC/sample_ibc.csv")
print(df.columns)

Index(['Unnamed: 0', 'sentence', 'label'], dtype='object')


In [10]:
from transformers import pipeline

# bart
pipe = pipeline(model="facebook/bart-large-mnli")

political_labels = ["Liberal", "Conservative", "Neutral"]

def classify_sentence(sentence):
    result = pipe(sentence, candidate_labels=political_labels)
    top_label = result["labels"][0]
    return top_label

df["predicted_label"] = df["sentence"].apply(classify_sentence)

correct_predictions = (df["predicted_label"] == df["label"]).sum()

total_sentences = len(df)
accuracy = correct_predictions / total_sentences

print(f"Correct predictions: {correct_predictions}")
print(f"Total sentences: {total_sentences}")
print(f"Accuracy: {accuracy:.2%}")

/opt/miniconda3/envs/cs375/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Correct predictions: 54
Total sentences: 150
Accuracy: 36.00%


In [11]:
# roberta and deberta, deberta out-performs roberta
# cross-encoder/nli-deberta-v3-base
from transformers import pipeline

# bert
pipe = pipeline(model="cross-encoder/nli-deberta-v3-base")

political_labels = ["Liberal", "Conservative", "Neutral"]

def classify_sentence(sentence):
    result = pipe(sentence, candidate_labels=political_labels)
    top_label = result["labels"][0]
    return top_label

df["predicted_label"] = df["sentence"].apply(classify_sentence)

correct_predictions = (df["predicted_label"] == df["label"]).sum()

total_sentences = len(df)
accuracy = correct_predictions / total_sentences

print(f"Correct predictions: {correct_predictions}")
print(f"Total sentences: {total_sentences}")
print(f"Accuracy: {accuracy:.2%}")


/opt/miniconda3/envs/cs375/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/miniconda3/envs/cs375/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:551: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Correct predictions: 59
Total sentences: 150
Accuracy: 39.33%
